# Evaluations on table parsing

We evaluate various parsing methods to extract tabular information from PDF files and analyze their influence on the downstream Q&A tasks. 

When measure the table-parsing methods, each question is paired with a PDF page contains the clue tables; doing so prevents inaccurate retrievals. 

We utilize the question set from PaperTab and the table-based questions from FinHybrid.

In [1]:
# Preparation
import sys
import os
from pathlib import Path

# Get the project root directory
root_dir = Path(os.path.abspath("")).resolve().parents[1]
sys.path.append(str(root_dir))
# Change the working directory to the project root
os.chdir(root_dir)

res_dir = f"experiment/parsing/res/"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)

In [2]:
# Experimental Configurations

DATASET_NAME_LIST = ["fin", "paper_tab"]
LOCAL_LLM_DICT = {"llama-8B": "meta-llama/Meta-Llama-3-8B-Instruct"}
LLM_LIST = ["gpt4", "llama-8B"]

LLM_LIST = LLM_LIST[:1]
DEMO_DOC_NUM = 1
DEMO_QA_NUM = 1

We evaluate several varied approaches of table parsing:
* **Raw text extraction**, which employs a PDF text extractor `PyPDF` to extract all the characters. 
  
*  Classic Computer Vision (**CV**) based approach, which often performs layout detection and OCR extraction at the same time. We use `Unstructured` library to use Yolox, Tesseract  and TableTransformer models together. 
  
*  **CV + LLM** method, which further employs an LLM to transform the outputs of (2) into Markdown tables. 
  
*  For the advanced multi-modal approach, we employ the latest **GPT-4-Omni** to convert image-based
document tables into Markdown format. 

* The FinHybrid dataset provides the verified **well-parsed** tables, which serve as the parsing ground truth.

Detailded parsing strategies are implemented in [uda/utils/parsing_exp.py](../../uda/utils/parsing_exp.py), and the following codes just call the encapsulated functions from it.

You need to install the [unstructured](https://docs.unstructured.io/open-source/installation/full-installation) library to run the CV and CV-LLM strategies.

We use the **AzureOpenAI-API**  as the interface for accessing GPT-4 and GPT-4o models. Users should set up the gpt-service with their own api-key and endpoint in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). 

If you want to use **other alternative platforms**, the following `call_gpt()` function and the [get_omni_table](../../uda/utils/parsing_exp.py#L185) function can be replaced.

In [3]:
from uda.utils.parsing_exp import get_fin_context, get_paper_context
from uda.utils import preprocess as pre
import pandas as pd
from uda.utils import llm
from uda.utils import inference
import json

strategies = ["raw_extract", "cv", "cv_llm", "well_parsed", "omni"]

for LLM_MODEL in LLM_LIST:
    for DATASET_NAME in DATASET_NAME_LIST:
        for strategy in strategies:
            if strategy == "well_parsed" and DATASET_NAME == "paper_tab":
                print(f"There is no well-parsed data for paper_tab dataset. Skip the strategy.")
                continue
            print(f"=== 🌟 Start {DATASET_NAME} on {LLM_MODEL} with parsing strategy {strategy} ===")
            res_file = os.path.join(res_dir, f"{DATASET_NAME}_{LLM_MODEL}_{strategy}.jsonl")

            # If use the local LLM, initialize the model
            if LLM_MODEL in LOCAL_LLM_DICT:
                llm_name = LOCAL_LLM_DICT[LLM_MODEL]
                llm_service = inference.LLM(llm_name)
                llm_service.init_llm()

            # Load the benchmark data
            bench_json_file = pre.meta_data[DATASET_NAME]["bench_json_file"]
            with open(bench_json_file, "r") as f:
                bench_data = json.load(f)

            # Run experiments on the demo docs
            doc_list = list(bench_data.keys())
            for doc in doc_list[:DEMO_DOC_NUM]:
                pdf_path = pre.get_example_pdf_path(DATASET_NAME, doc)
                if pdf_path is None:
                    continue
                for qa_item in bench_data[doc][:DEMO_QA_NUM]:
                    question = qa_item["question"]
                    # Parse the tables from the document and get the context
                    if DATASET_NAME == "fin":
                        context = get_fin_context(qa_item, strategy, pdf_path)
                    elif DATASET_NAME == "paper_tab":                            
                        context = get_paper_context(qa_item, strategy, pdf_path)
                    ## Show the context if needed
                    # print(context)
                    
                    # Create the prompt
                    llm_message = llm.make_prompt(question, context, DATASET_NAME, LLM_MODEL)
                    # Generate the answer
                    if LLM_MODEL in LOCAL_LLM_DICT:
                        response = llm_service.infer(llm_message)
                    elif LLM_MODEL == "gpt4":
                        # Set up with your own GPT4 service using environment variables
                        response = llm.call_gpt(messages=llm_message)
                        if response is None:
                            print("Make sure your gpt4 service is set up correctly.")
                            raise Exception("GPT4 service")

                    # log the results
                    res_dict = {
                        "model": LLM_MODEL,
                        "question": question,
                        "response": response,
                        "doc": doc,
                        "q_uid": qa_item["q_uid"],
                        "answers": qa_item["answers"],
                    }
                    print(res_dict)
                    with open(res_file, "a") as f:
                        f.write(json.dumps(res_dict) + "\n")

        print(f"======= Finish {DATASET_NAME} =======\n")


/root/miniconda3/envs/langchain/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda-11.8'


=== 🌟 Start fin on gpt4 with parsing strategy raw_extract ===


2024-07-05 13:03:44,980 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'what percentage of total long-term assets under supervision are comprised of fixed income in 2015?', 'response': 'The total long-term assets under supervision in 2015 were $922 billion, and the fixed income assets under supervision were $530 billion.\n\nTo calculate the percentage of fixed income assets in the total long-term assets under supervision for 2015:\n\n( Fixed Income / Total Long-Term Assets ) * 100\n( $530 billion / $922 billion ) * 100 = 57.48%\n\nThe answer is: 57.48%', 'doc': 'GS_2016', 'q_uid': 'GS/2016/page_79.pdf-3', 'answers': {'str_answer': '57%', 'exe_answer': 0.57484}}
=== 🌟 Start fin on gpt4 with parsing strategy cv ===


2024-07-05 13:03:46,174 - INFO - Reading PDF for file: ./experiment/parsing/parsing_tmp/fin_tmp.pdf ...
2024-07-05 13:03:46,342 - INFO - Detecting page elements ...
2024-07-05 13:03:47,609 - INFO - Processing entire page OCR with tesseract...
2024-07-05 13:03:50,660 - INFO - Loading the table structure model ...
2024-07-05 13:03:51,265 - INFO - Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)
2024-07-05 13:03:51,482 - INFO - [timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expec

...

{'model': 'gpt4', 'question': 'What baselines did they consider?', 'response': 'The answer is: off-the-shelf PDTB parsers, linear SVM, RBF SVM, Random forest, and LSTM classifiers.', 'doc': '1809.01202', 'q_uid': '4cbe5a36b492b99f9f9fea8081fe4ba10a7a0e94', 'answers': [{'answer': 'state-of-the-art PDTB taggers', 'type': 'extractive'}, {'answer': 'Linear SVM, RBF SVM, and Random Forest', 'type': 'abstractive'}]}
=== 🌟 Start paper_tab on gpt4 with parsing strategy cv_llm ===


2024-07-05 13:05:05,894 - INFO - Reading PDF for file: ./experiment/parsing/parsing_tmp/paper_tmp.pdf ...
2024-07-05 13:05:06,010 - INFO - Detecting page elements ...
2024-07-05 13:05:07,258 - INFO - Processing entire page OCR with tesseract...
2024-07-05 13:05:10,210 - INFO - Processing entire page OCR with tesseract...
2024-07-05 13:05:10,395 - INFO - padding image by 20 for structure detection
2024-07-05 13:05:10,875 - INFO - Processing entire page OCR with tesseract...
2024-07-05 13:05:11,037 - INFO - padding image by 20 for structure detection
2024-07-05 13:05:11,778 - INFO - Processing entire page OCR with tesseract...
2024-07-05 13:05:11,944 - INFO - padding image by 20 for structure detection
2024-07-05 13:05:23,645 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"
2024-07-05 13:05:27,245 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deploymen

{'model': 'gpt4', 'question': 'What baselines did they consider?', 'response': 'The answer is: off-the-shelf PDTB parsers, linear SVM, RBF SVM, Random forest, and LSTM classifiers.', 'doc': '1809.01202', 'q_uid': '4cbe5a36b492b99f9f9fea8081fe4ba10a7a0e94', 'answers': [{'answer': 'state-of-the-art PDTB taggers', 'type': 'extractive'}, {'answer': 'Linear SVM, RBF SVM, and Random Forest', 'type': 'abstractive'}]}
There is no well-parsed data for paper_tab dataset. Skip the strategy.
=== 🌟 Start paper_tab on gpt4 with parsing strategy omni ===


2024-07-05 13:05:41,441 - INFO - HTTP Request: POST https://yulong-eu2.openai.azure.com//openai/deployments/yulong-4o/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"
2024-07-05 13:05:43,902 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'What baselines did they consider?', 'response': 'The answer is: (Biran and McKeown, 2015), (Lin et al., 2014), Linear SVM, RBF SVM, Random Forest, LSTM.', 'doc': '1809.01202', 'q_uid': '4cbe5a36b492b99f9f9fea8081fe4ba10a7a0e94', 'answers': [{'answer': 'state-of-the-art PDTB taggers', 'type': 'extractive'}, {'answer': 'Linear SVM, RBF SVM, and Random Forest', 'type': 'abstractive'}]}
======= Finish paper_tab =======



### Evaluate the parsing results

In [4]:
dataset_name="fin"
llm_model="gpt4"
parsing_strategy="raw_extract"
res_file_name=f"experiment/parsing/res/{dataset_name}_{llm_model}_{parsing_strategy}.jsonl"

from uda.eval.my_eval import eval_from_file
eval_from_file(dataset_name, res_file_name)

Exact-match accuracy: 100.00
